# Google Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install

In [2]:
!pip3 install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00


# 5.3.2 Causal LM

### 5.3.2.1 모델

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)
model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

### 5.3.2.2 데이터셋 구조

In [5]:
from datasets import load_dataset

split_dict = {
    "train": "train[:8000]",
    "test": "train[8000:10000]",
    "unused": "train[10000:]",
}
dataset = load_dataset("heegyu/kowikitext", split=split_dict)
del dataset["unused"]
dataset

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for heegyu/kowikitext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/heegyu/kowikitext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'revid', 'url', 'title', 'text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['id', 'revid', 'url', 'title', 'text'],
        num_rows: 2000
    })
})

In [6]:
tokenized_dataset = dataset.map(
    lambda batch: tokenizer([f"{ti}\n{te}" for ti, te in zip(batch["title"], batch["text"])]),
    batched=True,
    num_proc=2,
    remove_columns=dataset['train'].column_names,
)
tokenized_dataset

Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [8]:
max_length = 512
def group_texts(batched_sample):
    sample = {k: v[0] for k, v in batched_sample.items()}

    if sample['input_ids'][-1] != tokenizer.eos_token_id:
        for k in sample.keys():
            sample[k].append(
                tokenizer.eos_token_id if k == "input_ids" else sample[k][-1]
            )

    result = {k: [v[i: i + max_length] for i in range(0, len(v), max_length)] for k, v in sample.items()}
    return result

grouped_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1,
    num_proc=2,
)
print(len(grouped_dataset['train'][0]['input_ids']))
print(grouped_dataset)

Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

512
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 18365
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2400
    })
})


In [9]:
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
sample = collator([grouped_dataset['train'][i] for i in range(1)])

### 5.3.2.4 문장 생성

In [13]:
sample = collator([grouped_dataset['train'][3]])

In [11]:
from transformers import GenerationConfig

gen_cfg = GenerationConfig(
    max_new_tokens=100,
    do_sample=True,
    temperature=1.2,
    top_k=50,
    top_p=0.95
)
outputs = model.generate(**sample, generation_config=gen_cfg)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result[0])

수학
수학(數學,, 줄여서 math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 다만 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.
수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 다른 여러 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.
수학을 의미하는 매스매틱스(mathematics)라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어'máthēma'(μθμα) 및 그 활용형 mathēmatikós(μαθματι)에서 유래되었다.
역사.
역사적으로 고대부터 현대에 이르기까지 문명에 필수적인 건축, 천문학, 정치, 상업 등에 수학적 개념들이 응용되어 왔다. 교역·분배·과세 등 인류의 사회 생활에 필요한 모든 계산에 수학이 관여해 왔고, 농경 생활에 필수적인 천문 관측과 달력의 제정, 토지의 측량 또한 수학이 직접적으로 사용된 분야이다. 고대 수학을 크게 발전시킨 문명으로는 이집트, 인도, 중국, 그리스 등이 있다.
특히 고대 그리스 문명에서는 처음으로 방정식에서 변수를 문자로 쓰는 등 추상화가 발전하였고 유클리드의 원론에서는 최초로 엄밀한 논증에 근거한 수학이 나타난다. 수학의 발전은 이후로도 계속되어 16세기의 르네상스에 이

# 5.3.3 Question Ansering

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)
model = AutoModelForQuestionAnswering.from_pretrained("skt/kogpt2-base-v2")
model

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForQuestionAnswering(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (qa_outputs): Linear(in_features=768, out_features=2, bias=True)
)

In [15]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        context_start = sequence_ids.index(1)
        context_end = sequence_ids.index(None) - 1 if None in sequence_ids else (len(sequence_ids) - 1)

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

Map:   0%|          | 0/17554 [00:00<?, ? examples/s]

Map:   0%|          | 0/5841 [00:00<?, ? examples/s]

In [16]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()
batch = data_collator([tokenized_dataset["train"][i] for i in range(10)])
batch

{'input_ids': tensor([[ 9233, 20493,  9032,  ...,     3,     3,     3],
         [28567, 15263, 15755,  ..., 40714,  9045, 12446],
         [21066,  8745,  9462,  ..., 12446, 42608,  8563],
         ...,
         [10528,   425, 10355,  ...,     3,     3,     3],
         [ 9150,  8160,  7109,  ...,     3,     3,     3],
         [ 9751,  9134, 35894,  ...,     3,     3,     3]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'start_positions': tensor([215,  24,   0,  52,  58,  66, 200, 311, 297, 279]),
 'end_positions': tensor([217,  26,   0,  53,  63,  71, 208, 317, 302, 283])}

In [17]:
with torch.no_grad():
  outputs = model(**batch)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = batch["input_ids"][0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

''

# 5.3.4 Sequence Classification

## 5.3.4.1 모델

In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)
model = AutoModelForSequenceClassification.from_pretrained("skt/kogpt2-base-v2", num_labels=2)
model

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

## 5.3.4.2 데이터셋 구조

In [19]:
from datasets import load_dataset

dataset = load_dataset("klue", "sts")

def process_data(batch):
  result = tokenizer(batch["sentence1"], text_pair=batch["sentence2"])
  result["labels"] = [x["binary-label"] for x in batch["labels"]]
  return result

dataset = dataset.map(process_data, batched=True, remove_columns=dataset["train"].column_names)

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

Map:   0%|          | 0/11668 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

In [20]:
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(tokenizer)
batch = collator([dataset["train"][i] for i in range(4)])

In [23]:
with torch.no_grad():
  logits = model(**batch).logits

logits

tensor([[-1.2254, -1.0138],
        [-0.7855, -1.0037],
        [-0.4224, -0.0551],
        [-1.4790,  0.8001]])

## 5.3.4.3 평가 매트릭

In [24]:
import evaluate

f1 = evaluate.load("f1")
f1.compute(predictions=logits.argmax(-1), references=batch['labels'], average='micro')

{'f1': 0.5}